In [1]:
!uv add evaluate==0.4.1
!uv add accelerate>=0.26.0

Resolved 154 packages in 4ms
Uninstalled 1 package in 47ms
Installed 1 package in 86ms                                 
 - torch==1.13.1
 + torch==2.0.0
Resolved 154 packages in 4ms
Audited 101 packages in 0.03ms


# Load Base Library

In [1]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import pandas as pd
import evaluate
import accelerate
import datasets

/home/nghiapd/Code/sum-med/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Check Lib Version

In [2]:
print(evaluate.__version__)
print(datasets.__version__)
# print(transformers.__version__)
print(accelerate.__version__)

0.4.1
2.0.0
1.0.1


# Data process

In [3]:
import pandas as pd

df_train = pd.read_csv("medical-summarization/train.tsv", sep="\t", header=None, names=["text", "summary"])
df_val   = pd.read_csv("medical-summarization/val.tsv", sep="\t", header=None, names=["text", "summary"])
df_test  = pd.read_csv("medical-summarization/test.tsv", sep="\t", header=None, names=["text", "summary"])


In [4]:
df_train = df_train.dropna()
df_train["text"] = df_train["text"].astype(str)
df_train["summary"] = df_train["summary"].astype(str)

In [5]:
df_val = df_val.dropna()
df_val["text"] = df_val["text"].astype(str)
df_val["summary"] = df_val["summary"].astype(str)

In [6]:
df_test = df_test.dropna()
df_test["text"] = df_test["text"].astype(str)
df_test["summary"] = df_test["summary"].astype(str)

In [7]:
print(df_train.dtypes, df_val.dtypes, df_test.dtypes)

text       object
summary    object
dtype: object text       object
summary    object
dtype: object text       object
summary    object
dtype: object


In [8]:
df_train

,text,summary
0,Cho em hỏi . Năm nay em 18 rồi mà vẫn bị hẹp v...,Dương vật bị hẹp và cong sang trái
1,Chào bác sĩ ! Tôi bị viêm dạ dày và trào ngược...,Bị viêm dạ dày và trào ngược thực quản ăn sữa ...
2,Chào bác sĩ . Tôi có một câu hỏi rất thắc mắc ...,Thuốc tránh thai ảnh hưởng đến lạc nội mạc tử ...
3,Chào bác sĩ ! Cách đây 2 năm em có đi cắt bao ...,"Cắt bao quy đầu có ảnh hưởng đến "" chuyện ấy "" ?"
4,Chào bác sĩ ! Em đã mổ nội soi đường tiết niệu...,Đi tiểu ra máu và đái rắt sau khi mổ nội soi đ...
...,...,...
10616,Kính gửi bác sĩ ! Hôm trước em có hỏi hiện tượ...,Trẻ 4 tháng bú mẹ 13 ngày vẫn chưa đi ngoài
10617,"Xin chào bác sĩ , tôi bị đau đầu sau gáy , có ...","Đau đầu , suy nghĩ lung tung dấu hiệu bệnh gì ?"
10618,"Bé em nay được 2 tháng , nửa tháng nay bé cứ b...","Bé đã ợ hơi sau bú nhưng vẫn bị đầy hơi , chướ..."
10619,"Bác sĩ ơi ! Bé nhà em được 4 tháng 20 ngày , b...",Bé bị sụt kg


In [9]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(df_train)
val_dataset   = Dataset.from_pandas(df_val)
test_dataset  = Dataset.from_pandas(df_test)

In [10]:
train_dataset[0]

{'text': 'Cho em hỏi . Năm nay em 18 rồi mà vẫn bị hẹp với lại dương vật của em bị cong sang trái 30-35 độ thì có bị sao không ạ ? Nguyễn Hậu ( 2001 ) Trả lời : Để biết chính xác tình trạng dương vật cong và hẹp của em nguyên nhân do đâu và có biện pháp điều trị phù hợp , em nên đến khám thực tế với bác sĩ Nam học tại Bệnh viện Vinmec Hải Phòng nhé ! Bác sĩ Bùi Thị Thu - Khoa Sản Phụ Khoa - Bệnh viện Đa khoa Quốc tế Vinmec Hải Phòng',
 'summary': 'Dương vật bị hẹp và cong sang trái'}

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("models/bartpho-word-base")
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = tokenizer(examples["text"], max_length=max_input_length, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    inputs["labels"] = labels["input_ids"]
    return inputs

In [12]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val   = val_dataset.map(preprocess_function, batched=True)
tokenized_test  = test_dataset.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x715fa4469790> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
  0%|          | 0/11 [00:00<?, ?ba/s]/home/nghiapd/Code/sum-med/.venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
100%|██

In [13]:
tokenized_train[0]

{'text': 'Cho em hỏi . Năm nay em 18 rồi mà vẫn bị hẹp với lại dương vật của em bị cong sang trái 30-35 độ thì có bị sao không ạ ? Nguyễn Hậu ( 2001 ) Trả lời : Để biết chính xác tình trạng dương vật cong và hẹp của em nguyên nhân do đâu và có biện pháp điều trị phù hợp , em nên đến khám thực tế với bác sĩ Nam học tại Bệnh viện Vinmec Hải Phòng nhé ! Bác sĩ Bùi Thị Thu - Khoa Sản Phụ Khoa - Bệnh viện Đa khoa Quốc tế Vinmec Hải Phòng',
 'summary': 'Dương vật bị hẹp và cong sang trái',
 'input_ids': [0,
  1852,
  193,
  447,
  5,
  434,
  136,
  193,
  890,
  182,
  64,
  74,
  45,
  4838,
  15,
  44,
  4481,
  1889,
  7,
  193,
  45,
  5310,
  295,
  732,
  7637,
  1830,
  378,
  54,
  10,
  45,
  423,
  17,
  3628,
  114,
  3751,
  3885,
  20,
  3583,
  19,
  20600,
  278,
  27,
  483,
  55,
  159,
  2906,
  939,
  22366,
  4481,
  1889,
  5310,
  6,
  4838,
  7,
  193,
  856,
  1362,
  91,
  602,
  6,
  10,
  19289,
  6222,
  184,
  3348,
  10932,
  2288,
  4,
  193,
  77,
  30,
  162

# Load Model

In [14]:
model_checkpoint = "models/bartpho-word-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Data Collector

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Create Metrics

In [16]:
import evaluate

rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
sacrebleu = evaluate.load("sacrebleu")
meteor = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to /home/nghiapd/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/nghiapd/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/nghiapd/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [17]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Làm sạch để tránh lỗi float
    decoded_preds = [str(p).strip() for p in decoded_preds]
    decoded_labels = [str(l).strip() for l in decoded_labels]

    # ROUGE
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {
        "rouge1": rouge_result["rouge1"],
        "rouge2": rouge_result["rouge2"],
        "rougeL": rouge_result["rougeL"],
    }

    # BLEU
    bleu_result = bleu.compute(predictions=decoded_preds, references=[[ref] for ref in decoded_labels])
    result["bleu"] = bleu_result["bleu"]

    # SacreBLEU
    sacrebleu_result = sacrebleu.compute(predictions=decoded_preds, references=[[ref] for ref in decoded_labels])
    result["sacrebleu"] = sacrebleu_result["score"]

    # METEOR
    meteor_result = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    result["meteor"] = meteor_result["meteor"]

    return {k: round(v, 4) for k, v in result.items()}

# Find Optimize Learning Rate 

- ##### /utils/find_best_lr.py

# Config Train

In [18]:
training_args = Seq2SeqTrainingArguments(
    output_dir="models/bartpho-word-base",
    evaluation_strategy="epoch",
    learning_rate=0.0008317637839354575,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=100,
    predict_with_generate=True,
    fp16=True,
)

/home/nghiapd/Code/sum-med/.venv/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_2419948/3690746773.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


# Train

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: 123baconsau6969 (nghiapd) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Bleu,Sacrebleu,Meteor
1,3.383400,3.278511,0.308300,0.060500,0.240100,0.012500,1.246600,0.105400
2,2.910000,3.099079,0.324500,0.072100,0.253100,0.030400,3.041800,0.131200
3,2.669800,3.055013,0.342000,0.068600,0.258100,0.024100,2.405500,0.121400
4,2.565100,3.022415,0.331300,0.069700,0.260200,0.028900,2.888500,0.128900
5,2.470100,3.025687,0.334100,0.074200,0.262200,0.024700,2.465600,0.122600
6,2.347800,3.080255,0.351100,0.079000,0.267400,0.023300,2.333600,0.138300
7,2.347200,3.059053,0.306700,0.067400,0.240300,0.022300,2.229400,0.114900
8,2.314100,3.030810,0.341800,0.070000,0.260700,0.028200,2.816100,0.138000
9,2.340700,3.072769,0.339400,0.073800,0.262200,0.028800,2.884500,0.138000
10,2.314500,3.070162,0.362200,0.084500,0.275400,0.031300,3.129400,0.143900


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr